## Construct the Burrows-Wheeler Transform of a String
[Rosalind BA9I](http://rosalind.info/problems/ba9i/)

In [124]:
def rotations(t):
    ''' Return list of rotations of input string t '''
    tt = t * 2
    return [ tt[i:i+len(t)] for i in range(0, len(t)) ]

def bwm(t):
    ''' Return lexicographically sorted list of t’s rotations '''
    return sorted(rotations(t))

def bwtViaBwm(t):
    ''' Given T, returns BWT(T) by way of the BWM '''
    return ''.join(map(lambda x: x[-1], bwm(t)))

In [125]:
t = 'abaaba$'
b = bwtViaBwm(t)
b

'abba$aa'

## Reconstruct a String from its Burrows-Wheeler Transform
[Rosalind BA9J](http://rosalind.info/problems/ba9j/)

In [126]:
def rankBwt(bw):
    ''' Given BWT string bw, return parallel list of B-ranks.  Also
        returns tots: map from character to # times it appears. '''
    tots = dict()
    ranks = []
    for c in bw:
        if c not in tots: tots[c] = 0
        ranks.append(tots[c])
        tots[c] += 1
    return ranks, tots

In [127]:
ranks, tots = rankBwt(b)

In [128]:
tots

{'a': 4, 'b': 2, '$': 1}

In [129]:
z = zip(b, ranks)
list(z)

[('a', 0), ('b', 0), ('b', 1), ('a', 1), ('$', 0), ('a', 2), ('a', 3)]

In [130]:
def firstCol(tots):
    ''' Return map from character to the range of rows prefixed by
        the character. '''
    first = {}
    totc = 0
    for c, count in sorted(tots.items()):
        first[c] = (totc, totc + count)
        totc += count
    return first

In [131]:
firstCol(tots)

{'$': (0, 1), 'a': (1, 5), 'b': (5, 7)}

In [132]:
# confirm that the representation of the first column above is sensible
print('\n'.join(bwm(t)))

$abaaba
a$abaab
aaba$ab
aba$aba
abaaba$
ba$abaa
baaba$a


In [133]:
def reverseBwt(bw):
    ''' Make T from BWT(T) '''
    ranks, tots = rankBwt(bw)
    first = firstCol(tots)
    rowi = 0 # start in first row
    t = '$' # start with rightmost character
    while bw[rowi] != '$':
        c = bw[rowi]
        t = c + t # prepend to answer
        # jump to row that starts with c of same rank
        rowi = first[c][0] + ranks[rowi]
    return t

In [134]:
reverseBwt(b)

'abaaba$'

## Generate the Last-to-First Mapping of a String
[Rosalind BA9K](http://rosalind.info/problems/ba9k/)

In [170]:
def findOccurences(BWT, char):
    ''' Find all occurences of character char in string BWT '''
    return [i for i, letter in enumerate(BWT) if letter == char]

def lastToFirst(BWT, idx):
    ''' Get position LastToFirst(i) in the first column in the BWM for a given BWT '''
    char = BWT[idx]
    occurence = findOccurences(BWT, char).index(idx)
    return sorted(BWT).index(char) + occurence

In [171]:
BWT = 'T$GACCA'
idx = 3
print(lastToFirst(BWT, idx))

1


## Implement BWMatching
[Rosalind BA9L](http://rosalind.info/problems/ba9l/)

    BWMATCHING(FirstColumn, LastColumn, Pattern, LastToFirst)
        top ← 0
        bottom ← |LastColumn| − 1
        while top ≤ bottom
            if Pattern is nonempty
                symbol ← last letter in Pattern
                remove last letter from Pattern
                if positions from top to bottom in LastColumn contain an occurrence of symbol
                    topIndex ← first position of symbol among positions from top to bottom in LastColumn
                    bottomIndex ← last position of symbol among positions from top to bottom in LastColumn
                    top ← LastToFirst(topIndex)
                    bottom ← LastToFirst(bottomIndex)
                else
                    return 0
            else
                return bottom − top + 1

In [178]:
def BWMMatching(BWT, pattern, l2f):
    top = 0
    bottom = len(BWT) -1
    
    while (top <= bottom):
        if pattern:
            symbol = pattern[-1]
            pattern = pattern[:-1]
            if symbol in BWT[top:bottom+1]:
                topIndex = BWT[top:bottom+1].find(symbol) + top
                bottomIndex = BWT[top:bottom+1].rfind(symbol) + top
                top = l2f[topIndex]
                bottom = l2f[bottomIndex]
            else:
                return 0
        else:
            return bottom - top + 1

In [179]:
BWT = 'TCCTCTATGAGATCCTATTCTATGAAACCTTCA$GACCAAAATTCTCCGGC'
patterns = 'CCT CAC GAG CAG ATC'
patterns = patterns.split()
l2f = []
for idx in range(0, len(BWT)):
        l2f.append(lastToFirst(BWT, idx))
print(" ".join(str(BWMMatching(BWT, pattern, l2f)) for pattern in patterns))

2 1 1 0 1
